#Laboratorio 10
###Integrantes: Franco Gonzalez - Martín Bravo - Iván Vidal
###Sección 2

In [1]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
!wget https://users.dcc.uchile.cl/~mtoro/cursos/BD2020/adult_with_pii.csv

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,016 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,498 kB]
Hit:13 https://ppa.launchpadcontent

In [2]:
%%sql
DROP TABLE IF EXISTS person;
CREATE TABLE person
(
  name varchar(255),
  dob date,
  ssn  varchar(50),
  zip integer,
  age integer,
  workclass varchar(255),
  fnlwgt integer,
  education varchar(255),
  education_num integer,
  marital_status varchar(255),
  occupation varchar(255),
  relationship varchar(255),
  race varchar(255),
  sex varchar(50),
  capital_gain integer,
  capital_loss integer,
  hours_per_week integer,
  country varchar(255),
  target varchar(255)
);
ALTER TABLE ONLY person ADD CONSTRAINT "person_key" PRIMARY KEY (name,ssn);

 * postgresql+psycopg2://@/postgres


""


Suponga que el gobierno de Estados Unidos cuenta con un base de datos de resultados del censo (la información de identificación personal (PII) es inventada)

In [3]:
%%sql
copy person(name, dob, ssn, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target) from
'/content/adult_with_pii.csv' delimiter ',' csv header;

 * postgresql+psycopg2://@/postgres


""


In [4]:
%sql SELECT * FROM person limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,2009-04-06,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,1951-09-16,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga que los identificadores explícitos de las personas son las columnas name y ssn (social security number).
Para proteger la privacidad de las personas, el gobierno de-identifica esta tabla borrando las columnas name y ssn.

In [5]:
%%sql
create view person_deid as select dob, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target from person;

 * postgresql+psycopg2://@/postgres


""


In [6]:
%sql select * from person_deid limit 5;

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,1967-09-07,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1988-06-07,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,1991-08-06,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,2009-04-06,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,1951-09-16,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga también que se filtró de una casa comercial, los nombres, fechas de nacimiento, ssn, código postal y edad de muchas personas. Nadie se preocupa por esto ya que no contiene información sensible (como el sueldo).
Crearemos una vista para representar esta data auxiliar.

In [7]:
%%sql
create view aux as select name, dob, ssn, zip, age from person;

 * postgresql+psycopg2://@/postgres


""


In [8]:
%sql select * from aux limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38
3,Dorry Poter,2009-04-06,659-57-4974,25503,53
4,Dick Honnan,1951-09-16,220-93-3811,75387,28


Para el lab, tenga en cuenta que puede guardar resultados de consultas en variables. Por ejemplo:

In [9]:
r = %sql select * from aux where name='Karrie Trusslove';
#podemos usar r como un dataframe
print("cantidad de registros:",len(r))
#usamos ['age'] para projectar la columna 'age', e iloc[0] para acceder a la fila número 0.
print("Edad de Karrie:", r['age'].iloc[0])
#puede construir consultas dinámicas usando interpolación como se describe a continuación:
def foo(tabla):
  r = %sql select * from $tabla
  return r
foo('person')

 * postgresql+psycopg2://@/postgres
cantidad de registros: 1
Edad de Karrie: 39
 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,2009-04-06,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,1951-09-16,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,Ardyce Golby,1961-10-29,212-61-8338,41328,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,Jean O'Connor,1952-06-28,737-32-2919,94735,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,Reuben Skrzynski,1966-08-09,314-48-0219,49628,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,Caye Biddle,1978-05-19,647-75-3550,8213,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


#Pregunta 1

Escriba una función que tome como argumento un conjunto de columnas y que realice un ataque de asociación a la tabla person_deid usando la tabla aux.
Muestre el resultado de al menos 3 ataques de asociación.


In [10]:
def ataque_asociacion(cols: list[str]):
  cond   = " where "
  select = ""
  for i, col in enumerate(cols):
    if( i>0 ):
      cond += " and "
      select += ", "
    cond += f" a.{col} = p.{col} "
    select += f" a.{col} "

  r = %sql select distinct count(*), $select from aux as a, person_deid as p $cond group by $select order by count(*);
  return r

### Ejemplos de 3 ataques de Asociación

In [11]:
ataque_asociacion(["zip"])

 * postgresql+psycopg2://@/postgres


,count,zip
0,1,4
1,1,12
2,1,16
3,1,18
4,1,23
...,...,...
27786,16,97618
27787,25,32527
27788,25,35283
27789,25,39163


In [12]:
ataque_asociacion(["zip","age"])

 * postgresql+psycopg2://@/postgres


,count,zip,age
0,1,4,55
1,1,12,24
2,1,16,59
3,1,17,32
4,1,17,52
...,...,...,...
32459,4,95437,34
32460,4,95676,36
32461,4,97763,49
32462,4,98303,39


In [13]:
ataque_asociacion(["zip","age","dob"])

 * postgresql+psycopg2://@/postgres


,count,zip,age,dob
0,1,4,55,1951-09-02
1,1,12,24,1977-11-23
2,1,16,59,1985-02-09
3,1,17,32,1985-07-18
4,1,17,52,1999-02-26
...,...,...,...,...
32556,1,99989,18,1986-03-16
32557,1,99989,20,1972-03-08
32558,1,99991,17,1986-03-11
32559,1,99995,31,1971-03-04


#  Pregunta 2

1.   Escriba cuántos individuos en esta base de datos son únicamente identificados por el código postal (zip). ¿A qué porcentaje del total corresponde? Recuerde que una persona es identificada únicamente por la combinación nombre y ssn.
2.   Haga lo mismo para el CId edad (age).
3.   ¿Qué puede concluir de este análisis?




### P2.1 zip

In [14]:
Total = ataque_asociacion(["zip"])
Datos_Totales = Total['count'].sum()
Identificados = (Total[Total['count'] == 1])["count"].sum()

print(f"La Cantidad de Individuos identificados con el código postal fue: {Identificados}")
print(f"El porcentaje del total es: {round(100*int(Identificados)/Datos_Totales, 2)}%")

 * postgresql+psycopg2://@/postgres
La Cantidad de Individuos identificados con el código postal fue: 23513
El porcentaje del total es: 54.44%


### P2.2 age

In [15]:
Total_age = ataque_asociacion(["age"])
Identificados_age = (Total_age[Total_age['count'] == 1])["count"].sum()

print(f"La Cantidad de Individuos identificados con la edad fue: {Identificados_age}")
print(f"El porcentaje del total es: {round(100*int(Identificados_age)/Datos_Totales, 4)}%")

 * postgresql+psycopg2://@/postgres
La Cantidad de Individuos identificados con la edad fue: 2
El porcentaje del total es: 0.0046%


### P2.3 Conclusión

Existen muchas más tuplas con atributo zip único que con atributo age único, ya que es mucho más común que, en una base de datos con muchas personas, existan 2 personas con la misma edad a que existan 2 personas con el mismo zip.



# Pregunta 3
El gobierno se da cuenta de que su intento de anonimización es muy vulnerable, por lo que solo permite a los analistas a realizar consultas de agregación sobre la tabla person que involucren muchas filas.
Obtenga el nivel educativo (education_num) del individuo llamado Ardyce Golby usando solo consultas de agregación que involucren al menos 1000 tuplas sobre la tabla *person*.

In [16]:
n_1 = %sql SELECT sum(education_num) FROM person;
n_2 = %sql SELECT sum(education_num) FROM person WHERE name<>'Ardyce Golby';
print(f"El nivel educativo de Ardyce Golby es {n_1['sum'].iloc[0] - n_2['sum'].iloc[0]}")

 * postgresql+psycopg2://@/postgres
 * postgresql+psycopg2://@/postgres
El nivel educativo de Ardyce Golby es 12


#Pregunta 4


1.   Suponga ahora que un atacante cuenta con data auxiliar que contiene el cuasi-identificador código postal (zip). Indique qué nivel de $k$-Anonimato cumple person_deid.
2.   Utilice alguna técnica de generalización y/o eliminación de valores atípicos sobre los cuasi-identificadores para lograr al menos $2$-anonimato.
3.   Repita los pasos 1 y 2 para el CId edad (age).
4.   Repita los pasos 1 y 2 para los CIds (zip, age). *Hint*: puede tratar de eliminar valores atípicos para la columna age.
5.   Concluya qué es lo que pasa cuando agregamos más CIds al análisis de $k$-anonimato.




### Resolución P4.1

In [39]:
p41 = %sql select zip, count(*) as count from person group by zip order by count asc;
print(f"\nEl nivel de anonimato es {p41['count'].iloc[0]} - anonimato\n")
%sql select zip, count(*) as count from person group by zip order by count asc limit 3;

 * postgresql+psycopg2://@/postgres

El nivel de anonimato es 1 - anonimato

 * postgresql+psycopg2://@/postgres


,zip,count
0,98561,1
1,95374,1
2,11233,1


### Resolución P4.2

In [44]:
p42 = %sql select (zip/100)*100 as zip2, count(*) as count from person group by zip2 order by count asc;
print(f"\nEl nivel de anonimato es {p42['count'].iloc[0]} - anonimato\n")
%sql select (zip/100)*100 as zip2, count(*) as c from person group by zip2 order by c asc limit 3;

 * postgresql+psycopg2://@/postgres

El nivel de anonimato es 16 - anonimato

 * postgresql+psycopg2://@/postgres


,zip2,c
0,71100,16
1,52700,17
2,43300,18


### Resolución P4.3

In [47]:
p43 = %sql select age, count(*) as count from person group by age order by count asc;
print(f"\nEl nivel de anonimato es {p43['count'].iloc[0]} - anonimato\n")
%sql select age, count(*) as count from person group by age order by count asc limit 3;

 * postgresql+psycopg2://@/postgres

El nivel de anonimato es 1 - anonimato

 * postgresql+psycopg2://@/postgres


,age,count
0,87,1
1,86,1
2,88,3


In [48]:
p43_ = %sql select (age/10)*10 as age2, count(*) as count from person group by age2 order by count asc;
print(f"\nEl nivel de anonimato es {p43_['count'].iloc[0]} - anonimato\n")
%sql select (age/10)*10 as age2, count(*) as count from person group by age2 order by count asc limit 3;


 * postgresql+psycopg2://@/postgres

El nivel de anonimato es 43 - anonimato

 * postgresql+psycopg2://@/postgres


,age2,count
0,90,43
1,80,78
2,70,508


### Resolución p4.4

In [55]:
p44 = %sql select age, zip, count(*) as count from person group by age, zip order by count asc;
print(f"\nEl nivel de anonimato es {p44['count'].iloc[0]} - anonimato\n")
%sql select age, zip, count(*) as count from person group by age, zip order by count asc limit 3;

 * postgresql+psycopg2://@/postgres

El nivel de anonimato es 1 - anonimato

 * postgresql+psycopg2://@/postgres


,age,zip,count
0,32,10616,1
1,25,94556,1
2,33,38153,1


In [54]:
p44_ = %sql select (age/25)*25 as age2, (zip/10000)*10000 as zip2, count(*) as count from person group by age2, zip2 order by count asc;
print(f"\nEl nivel de anonimato es {p44_['count'].iloc[0]} - anonimato\n")
%sql select (age/25)*25 as age2, (zip/10000)*10000 as zip2, count(*) as count from person group by age2, zip2 order by count asc limit 3;

 * postgresql+psycopg2://@/postgres

El nivel de anonimato es 19 - anonimato

 * postgresql+psycopg2://@/postgres


,age2,zip2,count
0,75,40000,19
1,75,50000,21
2,75,70000,23


##Conclusión P4.5
Al agregar mas CIds es mucho más fácil identificar a las personas debido a que al agregar más identificadores la probabilidad de que 2 personas tengan todos sus identificadores iguales va disminuyendo, en especial si esos identificadores pueden tener muchos valores posibles como en el caso del zip. Esto complica poder generalizar los valores atípicos para mantener su privacidad.

#Pregunta 5

Suponga que los CIds son edad y nivel de educación (age, education_num), y estado civil un dato sensible (marital_status).

1.   Escriba una consulta e indique si la tabla es $2$-diversa.
2.   Utilice generalización y/o eliminación de valores atípicos para lograr al menos $5$-diversidad.
3.   Para el resultado en (2), escriba la consulta e indique el nivel de $k$-anonimato que se logra. ¿Qué puede concluir de esto?
4.   (Opcional/Bonus) La **diversidad entrópica** de un grupo con el mismo CIds mide cuán uniformemente están distribuidos los valores privados de algún atributo $S$ en el grupo. Se define formalmente como
$$-\sum_{v\in S}\mathtt{P}(\mathit{cIds}, v)*\mathtt{log}(\mathtt{P}(\mathit{cIds}, v))$$ donde $\mathtt{P}(\mathit{cIds}, v)$ corresponde a la fracción de registros en el grupo $\mathit{cIds}$ con valor $v$ para el atributo $S$.
Por ejemplo, para la siguiente tabla donde la enfermedad es un dato sensible:

| Trabajo | Sexo | Edad | Enfermedad |
| --- | --- | --- | --- |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | VIH |
| Artista | Femenino | [35-35) | Resfrío |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |

La diversidad entrópica del grupo (Profesor, Masculino [35-40)) es de $-(\frac{2}{3}\mathtt{log}\frac{2}{3} + \frac{1}{3}\mathtt{log}\frac{1}{3}) = \mathtt{log}(1.9)$, y la diversidad entropica del grupo (Artista, Femenino, [35-35)) es $-(\frac{3}{4}\mathtt{log}\frac{3}{4} + \frac{1}{4}\mathtt{log}\frac{1}{4}) = \mathtt{log}(1.8)$.

Calcule la **diversidad entrópica** de cada grupo en (2), y concluya cuál es el conjunto de cids más y menos diversos.



###Resolución P5.1

In [66]:
p51 = %sql select age as age2, education_num as education_n, count(distinct marital_status) as count from person group by age2, education_n order by count asc;
print(f"\nEl nivel de diversidad es {p51['count'].iloc[0]} - diversa, es decir no cumple lo del enunciado\n")
%sql select age as age2, education_num as education_n, count(distinct marital_status) as count from person group by age2, education_n order by count asc limit 3;

 * postgresql+psycopg2://@/postgres

El nivel de diversidad es 1 - diversa, es decir no cumple lo del enunciado

 * postgresql+psycopg2://@/postgres


,age2,education_n,count
0,17,4,1
1,17,3,1
2,17,7,1


###Resolución P5.2

In [70]:
p52 = %sql select (age/20)*20 as age2, (education_num/10)*10 as education_n, count(distinct marital_status) as count from person group by age2, education_n order by count asc;
print(f"\nEl nivel de diversidad es {p52['count'].iloc[0]} - diversa, cumple con lo pedido\n")
%sql select (age/20)*20 as age2, (education_num/10)*10 as education_n, count(distinct marital_status) as count from person group by age2, education_n order by count asc;

 * postgresql+psycopg2://@/postgres

El nivel de diversidad es 5 - diversa, cumple con lo pedido

 * postgresql+psycopg2://@/postgres


,age2,education_n,count
0,0,10,5
1,80,10,5
2,60,10,6
3,80,0,6
4,0,0,7
5,60,0,7
6,40,10,7
7,20,0,7
8,20,10,7
9,40,0,7


###Resolución P5.3

In [79]:
p53 = %sql select (age/20)*20 as a, (education_num/10)*10 as e, count(marital_status) as c from person group by a, e order by c asc;
print(f"\nEl nivel de anonimato alcanzado es {p53['c'].iloc[0]} - anonimato\n")
%sql select (age/20)*20 as a, (education_num/10)*10 as e, count(marital_status) as c from person group by a, e order by c asc;

 * postgresql+psycopg2://@/postgres

El nivel de anonimato alcanzado es 51 - anonimato

 * postgresql+psycopg2://@/postgres


,a,e,c
0,80,10,51
1,80,0,70
2,0,10,455
3,60,10,1121
4,0,0,1202
5,60,0,1402
6,40,0,4967
7,40,10,6626
8,20,0,7113
9,20,10,9554


# Pregunta 6

1.   Escriba una consulta que responda a "¿cuántas personas nunca se han casado?"
2.   La sensibilidad de una consulta se puede calcular como: cuánto es lo máximo que puede variar el resultado de mi consulta si saco/agrego un único individuo de la base de datos. Escriba y justifique la sensibilidad de la consulta de (1).
3.   Use la distribución de laplace para contestar (1) de manera diferencialmente privada, con epsilon = 0.1.
    
    *Hint*: Para generar ruido aleatorio puede utilizar la función laplace de numpy:
```
import numpy as np
np.random.laplace(loc=0, scale=s)
```
y para aplicar una función a todos los elementos de un resultado de un consulta (dataframe) puede usar la función apply:
```
r = %sql ...
r.apply(lambda x: ...)
```
4.   Escriba una consulta $0.1$-diferencialmente privada que calcule la cantidad de personas agrupadas por edad y estado civil. Asegúrese que su respuesta no contenga cantidades negativas ni decimales (puede modificar los valores después de agregarle ruido).

    *Hint*: Use la función map para aplicar una función a todos los elementos de una columna de la siguiente forma:
```
r = %sql ...
r['nombre_columna_nueva'] = r['nombre_columna'].map(lambda x: ...)
```

5.    Comente acerca de la utilidad y la privadcidad de la respuesta para grupos muy chicos vs grupos grandes después de agregar ruido.


###Resolución p6.1

In [80]:
nc = %sql select count(*) as c from person where  marital_status='Never-married' order by c desc limit 1;
print(f"La cantidad de personas que nunca se han casado son {nc['c'].iloc[0]}")

 * postgresql+psycopg2://@/postgres
La cantidad de personas que nunca se han casado son 10683


### P6.2

Lo máximo que puede variar la consulta al sacar o agregar una persona es 1, ya que la consulta simplemente cuenta las personas que cumplen con una condición, por lo que si se saca o se agrega una persona que justo cumple con la condición entonces la consulta solo puede aumentar o disminuir en 1.

###Resolución P6.3


In [98]:
import numpy as np

r = %sql select count(*) as count from person where marital_status='Never-married';
a = r.apply(lambda x: x + np.random.laplace(loc=0, scale=1/0.1))

print(f"La cantidad de personas que nunca se han casado son {a['count'].iloc[0]}")

 * postgresql+psycopg2://@/postgres
La cantidad de personas que nunca se han casado son 10687.59696885507


###Resolución P6.4

In [105]:
r = %sql select count(*) as count from person group by age, marital_status order by count asc;
r["count"] = r["count"].map(lambda x: int(max(x+np.random.laplace(loc=0, scale=1/0.1), 0)))
r

 * postgresql+psycopg2://@/postgres


,count
0,0
1,7
2,0
3,3
4,0
...,...
391,652
392,666
393,676
394,660


###Resolución P6.5

Para grupos muy pequeños, la función de ruido puede generar números muy alejados de la realidad, esto puede dificultar los análisis (datos menos útiles) y la privacidad que se gana no es tanta ya que siguen siendo grupos pequeños. Para el caso de grupos grandes, se gana privacidad y no se pierde tanta información (datos más útiles), ya que al ser números grandes las diferencias pequeñas no deberían afectar mucho al análisis.